In [35]:
import pandas as pd
import time
import plotly.graph_objects as go

 tiene sentido entrenar un modelo de serie temporal para predecir los gastos de una empresa por categoría de gasto utilizando datos de múltiples empresas. Este enfoque se conoce como análisis de series temporales multivariadas y puede ser útil en situaciones donde deseas aprovechar la información de múltiples fuentes para mejorar las predicciones.

Aquí hay algunas razones por las cuales podría ser beneficioso utilizar datos de varias empresas para predecir los gastos por categoría de gasto:

Mayor cantidad de datos: Al combinar datos de múltiples empresas, puedes tener acceso a una mayor cantidad de datos, lo que puede ayudar a mejorar la precisión del modelo y a capturar una variedad más amplia de patrones y tendencias en los datos.
Mejor generalización: Al entrenar el modelo con datos de múltiples empresas, el modelo puede aprender patrones y tendencias generales que se aplican a una variedad de empresas en lugar de depender únicamente de los datos de una sola empresa. Esto puede ayudar a que el modelo generalice mejor a nuevas empresas o situaciones.
Captura de variabilidad: Las diferentes empresas pueden tener diferentes patrones de gasto debido a factores como el tamaño de la empresa, la industria, la ubicación geográfica, etc. Al entrenar el modelo con datos de múltiples empresas, puedes capturar una mayor variabilidad en los datos, lo que puede ayudar a mejorar la capacidad del modelo para adaptarse a diferentes escenarios.
Incorporación de efectos externos: Al utilizar datos de múltiples empresas, también puedes incorporar información sobre factores externos que pueden influir en los gastos, como condiciones económicas, tendencias del mercado, cambios regulatorios, etc. Esto puede ayudar a que el modelo capture mejor el contexto en el que se producen los gastos.
Es importante tener en cuenta que al entrenar un modelo de serie temporal con datos de múltiples empresas, es posible que necesites tener en cuenta las diferencias entre las empresas y aplicar técnicas adecuadas para manejar la heterogeneidad de los datos. Además, es fundamental asegurarse de que los datos se agreguen o normalicen correctamente para que sean comparables entre sí.

In [36]:
df = pd.read_csv("../data/datos.csv")

In [37]:
df = df[["cc_num", "category", "trans_date_trans_time", "amt"]].copy()

In [38]:
df["cc_num"].unique()

array([   4642255475285942, 4512828414983801773,      30270432095985,
          4364010865167176,    6011438889172900,    4586810168620942,
           344709867813900,    6538891242532018,    4904681492230012,
          6538441737335434], dtype=int64)

In [39]:
df["category"].unique()

array(['gas_transport', 'misc_net', 'grocery_net', 'grocery_pos',
       'misc_pos', 'travel', 'kids_pets', 'home', 'shopping_net',
       'food_dining', 'personal_care', 'health_fitness', 'entertainment',
       'shopping_pos'], dtype=object)

In [40]:
df['trans_date_trans_time'] = pd.to_datetime(df['trans_date_trans_time'])
# Extraer solo la hora y descartar la información de la fecha
df['hour'] = df['trans_date_trans_time'].dt.hour
# Extraer solo la fecha y descartar la información de la hora
df['trans_date_trans_time'] = df['trans_date_trans_time'].dt.date

In [41]:
import numpy as np

In [42]:
def sacar_x_y(lista, T):
    T = T # Segmentos
    X = list()
    Y = list()

    for t in range(len(lista) - T):

        x = lista[t : t + T]
        X.append(x)


        y = lista[t + T]
        Y.append(y)

    # Transformamos a np.array y ajustamos las dimensiones
    # Para entrar en el modelo debe de tener 3 dimensiones
    X = np.array(X).reshape(-1, T, 1)
    Y = np.array(Y)
    N = len(X)

    return X, Y, T

In [43]:
# Redes neuronales
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, SimpleRNN

# Métricas
#from sklearn.metrics import r2_score

In [44]:
import sys
from contextlib import redirect_stdout

In [45]:
df

,cc_num,category,trans_date_trans_time,amt,hour
0,4642255475285942,gas_transport,2019-01-01,102.13,0
1,4642255475285942,misc_net,2019-01-01,177.57,0
2,4512828414983801773,grocery_net,2019-01-01,16.16,0
3,30270432095985,misc_net,2019-01-01,3.66,1
4,4364010865167176,gas_transport,2019-01-01,61.99,2
...,...,...,...,...,...
43859,6538891242532018,home,2020-12-31,9.28,22
43860,6538441737335434,personal_care,2020-12-31,26.38,23
43861,6011438889172900,shopping_net,2020-12-31,410.05,23
43862,4642255475285942,misc_pos,2020-12-31,5.58,23


In [46]:
from datetime import datetime, timedelta

# Fecha inicial y final
fecha_inicial = datetime(2019, 1, 1)
fecha_final = datetime(2020, 12, 31)

# Crear la lista de fechas
lista_fechas = [fecha_inicial + timedelta(days=d) for d in range((fecha_final - fecha_inicial).days + 1)]


# ----- SERIE TEMPORAL PARA PREDECIR GASTO DEL MES SIGUIENTE -----

In [47]:
import pickle

for i in df["cc_num"].unique():
    df_cuenta = df[df["cc_num"] == i]
    # Agrupar el DataFrame por 'fecha' y 'categoria', y luego sumar los montos
    df_cuenta = df_cuenta.groupby(['trans_date_trans_time'])['amt'].sum().reset_index()
    # Convertir la columna 'fecha' a tipo datetime si no lo está
    df_cuenta['trans_date_trans_time'] = pd.to_datetime(df_cuenta['trans_date_trans_time'])

    # Obtener las fechas que faltan en el DataFrame
    fechas_faltantes = set(lista_fechas) - set(df_cuenta['trans_date_trans_time'])

    # Crear un DataFrame con las fechas faltantes y un valor de 0
    df_faltantes = pd.DataFrame({'trans_date_trans_time': list(fechas_faltantes), 'amt': 0})

    # Concatenar el DataFrame original con el DataFrame de fechas faltantes
    df_completo = pd.concat([df_cuenta, df_faltantes])

    # Ordenar el DataFrame por la columna 'fecha'
    df_completo = df_completo.sort_values(by='trans_date_trans_time')

    print(f"----------------------------- Cuenta: {i} ---------------------------------------------------")

    train = df_completo["amt"][0:int(2*(df_cuenta.shape[0]/3))].to_list()
    test = df_completo["amt"][int(2*(df_cuenta.shape[0]/3)):].to_list()

    X_train, y_train, T = sacar_x_y(train, 10)
    X_test, y_test, T = sacar_x_y(test, 10)
    print(X_train.shape)
    print(y_train.shape)
    print(X_test.shape)
    print(y_test.shape)

    modelo = Sequential()

    modelo.add(Input(shape = (10, 1)))

    # Capa de entrada
    modelo.add(Dense(64, activation='relu'))

    # Capas ocultas
    modelo.add(Dense(128, activation='relu'))
    modelo.add(Dropout(0.2))  # Regularización con dropout para prevenir el sobreajuste
    modelo.add(Dense(64, activation='relu'))

    # Capa de salida
    modelo.add(Dense(1, activation='linear'))  # Capa de salida lineal para la regresión

    # Compilar el modelo
    modelo.compile(optimizer='adam', loss='mean_squared_error')

    # Crear un archivo para guardar el summary
    with open(f'../data/summary_{i}.txt', 'w') as f:
        # Redirigir la salida estándar hacia el archivo
        with redirect_stdout(f):
            # Imprimir el summary del modelo
            modelo.summary()

    modelo.compile(optimizer="adam", loss="mse", metrics=['mae', 'mape'])

    # Definir la devolución de llamada de EarlyStopping
    #early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

    history = modelo.fit(x = X_train,
                        y = y_train,
                        validation_data = (X_test, y_test),
                        #callbacks=[early_stopping],
                        epochs = 200)
    y_pred_test = modelo.predict(X_test)  # Esto te da las predicciones del modelo para los datos de entrenamiento
    print(y_pred_test.shape)
    print(y_test.shape)
    # Guardar el modelo en un archivo
    modelo.save(f'../data/modelo_{i}.keras')
    modelo.save(f'../data/modelo_{i}.h5')
    
    with open(f'../data/modelo_{i}.pkl', "bw") as file:
        pickle.dump(modelo, file)

    # modelo.save(f'../data/modelo_{i}')
    #joblib.dump(modelo, f'modelo_{i}.pkl')
            
    df_metricas = pd.DataFrame(columns=["mae", "loss", "val_mae", "val_loss"])
    #Calcular R^2 en el conjunto de entrenamiento
    #r2_train = r2_score(y_test.reshape(-1, 1), y_pred_test.reshape(-1, 1))
    # Historial de pérdida
    mae = history.history['mae']
    val_mae = history.history['val_mae']
    # Historial de pérdida
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    df_metricas["mae"] = mae
    df_metricas["val_mae"] = val_mae
    df_metricas["loss"] = loss
    df_metricas["val_loss"] = val_loss

    df_metricas.to_csv(f"../data/df_metricas_{i}.csv", index=False)
    

----------------------------- Cuenta: 4642255475285942 ---------------------------------------------------
(471, 10, 1)
(471,)
(240, 10, 1)
(240,)
Epoch 1/200
15/15 [==============================] - 2s 30ms/step - loss: 237647.8438 - mae: 293.0184 - mape: 870301568.0000 - val_loss: 300546.6562 - val_mae: 297.1041 - val_mape: 803964096.0000
Epoch 2/200
15/15 [==============================] - 0s 8ms/step - loss: 223066.7344 - mae: 273.8537 - mape: 2584661504.0000 - val_loss: 300877.2188 - val_mae: 303.3291 - val_mape: 649816896.0000
Epoch 3/200
15/15 [==============================] - 0s 8ms/step - loss: 223114.3906 - mae: 275.2005 - mape: 1857874816.0000 - val_loss: 302519.1250 - val_mae: 306.6068 - val_mape: 598159424.0000
Epoch 4/200
15/15 [==============================] - 0s 8ms/step - loss: 222717.4844 - mae: 273.8552 - mape: 2167764480.0000 - val_loss: 300672.5000 - val_mae: 303.0383 - val_mape: 652488320.0000
Epoch 5/200
15/15 [==============================] - 0s 9ms/step - lo

C:\Users\enaat\AppData\Roaming\Python\Python311\site-packages\keras\src\engine\training.py:3000: UserWarning:

You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.



----------------------------- Cuenta: 4512828414983801773 ---------------------------------------------------
(472, 10, 1)
(472,)
(239, 10, 1)
(239,)
Epoch 1/200
15/15 [==============================] - 2s 30ms/step - loss: 186085.1875 - mae: 272.9173 - mape: 1153256704.0000 - val_loss: 328654.7500 - val_mae: 322.1108 - val_mape: 1281855104.0000
Epoch 2/200
15/15 [==============================] - 0s 8ms/step - loss: 171218.1250 - mae: 243.5168 - mape: 1697657600.0000 - val_loss: 327399.5312 - val_mae: 322.7352 - val_mape: 1053856512.0000
Epoch 3/200
15/15 [==============================] - 0s 8ms/step - loss: 166855.4688 - mae: 241.3745 - mape: 1747196416.0000 - val_loss: 326934.8438 - val_mae: 321.9356 - val_mape: 1139609472.0000
Epoch 4/200
15/15 [==============================] - 0s 8ms/step - loss: 166102.7344 - mae: 240.8391 - mape: 1863919872.0000 - val_loss: 326893.5312 - val_mae: 321.7876 - val_mape: 1155135360.0000
Epoch 5/200
15/15 [==============================] - 0s 7ms/s

C:\Users\enaat\AppData\Roaming\Python\Python311\site-packages\keras\src\engine\training.py:3000: UserWarning:

You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.



Epoch 1/200
15/15 [==============================] - 2s 27ms/step - loss: 192175.9219 - mae: 280.3759 - mape: 1199752960.0000 - val_loss: 351304.5000 - val_mae: 316.9238 - val_mape: 1356288512.0000
Epoch 2/200
15/15 [==============================] - 0s 8ms/step - loss: 167426.2188 - mae: 246.9934 - mape: 3499568384.0000 - val_loss: 350742.7500 - val_mae: 317.3470 - val_mape: 1324874752.0000
Epoch 3/200
15/15 [==============================] - 0s 8ms/step - loss: 165572.8594 - mae: 245.4689 - mape: 2823719168.0000 - val_loss: 349688.9062 - val_mae: 320.3877 - val_mape: 1176790912.0000
Epoch 4/200
15/15 [==============================] - 0s 8ms/step - loss: 165819.2344 - mae: 245.3574 - mape: 2865559296.0000 - val_loss: 350864.3438 - val_mae: 317.0117 - val_mape: 1341091456.0000
Epoch 5/200
15/15 [==============================] - 0s 9ms/step - loss: 165058.2656 - mae: 244.7464 - mape: 3110412800.0000 - val_loss: 349636.5938 - val_mae: 319.3163 - val_mape: 1218019328.0000
Epoch 6/200
15

...

C:\Users\enaat\AppData\Roaming\Python\Python311\site-packages\keras\src\engine\training.py:3000: UserWarning:

You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.



Epoch 1/200
15/15 [==============================] - 2s 27ms/step - loss: 239854.4062 - mae: 288.3393 - mape: 877011968.0000 - val_loss: 362700.5625 - val_mae: 342.4626 - val_mape: 590615552.0000
Epoch 2/200
15/15 [==============================] - 0s 8ms/step - loss: 224189.2344 - mae: 268.2639 - mape: 1776440448.0000 - val_loss: 363839.2188 - val_mae: 344.3334 - val_mape: 532221376.0000
Epoch 3/200
15/15 [==============================] - 0s 8ms/step - loss: 223911.8281 - mae: 269.4828 - mape: 1713113600.0000 - val_loss: 363697.4688 - val_mae: 344.1716 - val_mape: 534796032.0000
Epoch 4/200
15/15 [==============================] - 0s 8ms/step - loss: 225992.8125 - mae: 268.1512 - mape: 1939501696.0000 - val_loss: 363185.7812 - val_mae: 343.5240 - val_mape: 550251584.0000
Epoch 5/200
15/15 [==============================] - 0s 8ms/step - loss: 224381.5156 - mae: 269.2935 - mape: 1613725696.0000 - val_loss: 364981.0312 - val_mae: 345.6565 - val_mape: 502577696.0000
Epoch 6/200
15/15 [=

C:\Users\enaat\AppData\Roaming\Python\Python311\site-packages\keras\src\engine\training.py:3000: UserWarning:

You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.



Epoch 1/200
15/15 [==============================] - 1s 22ms/step - loss: 985584.3125 - mae: 364.7755 - mape: 3128475648.0000 - val_loss: 612143.0625 - val_mae: 488.8399 - val_mape: 94.5596
Epoch 2/200
15/15 [==============================] - 0s 7ms/step - loss: 966723.5625 - mae: 375.4326 - mape: 1944907136.0000 - val_loss: 632882.0000 - val_mae: 505.4579 - val_mape: 93.2839
Epoch 3/200
15/15 [==============================] - 0s 11ms/step - loss: 967005.3125 - mae: 374.4239 - mape: 1895873280.0000 - val_loss: 613812.5625 - val_mae: 490.1282 - val_mape: 94.3529
Epoch 4/200
15/15 [==============================] - 0s 7ms/step - loss: 971634.0625 - mae: 368.7301 - mape: 2180038144.0000 - val_loss: 625241.0000 - val_mae: 499.2898 - val_mape: 93.5368
Epoch 5/200
15/15 [==============================] - 0s 7ms/step - loss: 968171.6250 - mae: 374.0563 - mape: 2038254720.0000 - val_loss: 624252.5625 - val_mae: 498.4812 - val_mape: 93.5687
Epoch 6/200
15/15 [==============================] - 

C:\Users\enaat\AppData\Roaming\Python\Python311\site-packages\keras\src\engine\training.py:3000: UserWarning:

You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.



In [ ]:
#crear una lista de fechas
#comparo la columna fecha con la lista y veo cuales faltan
#a las que falten le añado un 0 
#ordenar el df por fecha

In [31]:
#añadir un apartado que sea la media de gastos diarios, mensual y semanal
#añadir grafica de barras por mes con los gastos de cada dia con los dias de colores
#grafico quesito semanal por categorias (modificar el total historico)
#modificar el total historico pasarlo de grafico quesito a barras mostrar top 5
#grafica acumulativa (cuanto dinero llevo gastado) por categoria o por suma de gastos diario

In [33]:
import plotly.express as px

El gráfico muestra el historial de tu pérdida de peso durante un entrenamiento. El eje X muestra la época, que es una unidad de tiempo utilizada para medir el progreso en el aprendizaje automático. El eje Y muestra la pérdida, que es una medida de qué tan bien está funcionando el modelo. Las dos líneas en el gráfico muestran la pérdida de entrenamiento y la pérdida de validación.

* **Pérdida de entrenamiento:** Disminuye rápidamente al comienzo del proceso de entrenamiento. Esto se debe a que el modelo está aprendiendo los patrones básicos de los datos. A medida que continúa el proceso, la pérdida de entrenamiento disminuye más lentamente porque el modelo aprende patrones más complejos.
* **Pérdida de validación:** Suele comenzar más alta que la pérdida de entrenamiento. Esto se debe a que el modelo no está tan familiarizado con los datos de validación como con los datos de entrenamiento. A medida que avanza el entrenamiento, la pérdida de validación disminuye. Sin embargo, es importante tener en cuenta que a veces puede comenzar a aumentar después de un cierto punto. Esto se debe a que el modelo se está sobreajustando a los datos de entrenamiento. El sobreajuste es un problema que ocurre cuando el modelo aprende demasiado bien los datos de entrenamiento y no puede generalizarse a datos nuevos.

**Rendimiento general:**

Se puede evaluar el rendimiento general del modelo observando la pérdida de validación. Si la pérdida de validación es baja, entonces el modelo está funcionando bien. Si la pérdida de validación es alta, entonces el modelo no está funcionando bien. 

**En el contexto del gráfico específico se pueden hacer las siguientes observaciones:**

* El modelo está funcionando bien. La pérdida de validación es relativamente baja y está disminuyendo.
* El modelo no se está sobreajustando a los datos de entrenamiento. La pérdida de validación no aumenta después de cierto punto.

**En general, el gráfico muestra que el modelo está funcionando bien y que no se está sobreajustando a los datos de entrenamiento.**

Espero que esta información sea útil. Avisame si tienes alguna otra pregunta.

### tengo un modelo que predice el gasto que tendrá un cliente el mes que viene, como creo ahora otro modelo que diga el porcentaje de ese gasto destinado a cada categoria?



Creación de un modelo para predecir el porcentaje de gasto por categoría
Para crear un modelo que predice el porcentaje de gasto en cada categoría para un cliente, puedes seguir estos pasos:

1. Recopilación de datos:

Necesitarás datos históricos del gasto de los clientes, desglosados ​​por categoría. Estos datos deben incluir el gasto total del cliente y el gasto en cada categoría individual.
Es importante que los datos sean precisos y completos, ya que la calidad de los datos afectará directamente el rendimiento del modelo.
2. Preparación de datos:

Los datos deben limpiarse y preprocesarse antes de poder usarse para entrenar el modelo. Esto puede implicar eliminar valores atípicos, manejar valores faltantes y normalizar los datos.
También es posible que debas crear características adicionales a partir de los datos existentes. Por ejemplo, puedes crear una característica que represente la proporción del gasto total gastado en cada categoría.
3. Selección del modelo:

Existen varios modelos de aprendizaje automático que se pueden utilizar para este problema. Algunos modelos populares incluyen regresión lineal, árboles de decisión y redes neuronales.
La mejor opción de modelo dependerá de los datos específicos y de los objetivos del proyecto.
4. Entrenamiento del modelo:

Una vez que hayas seleccionado un modelo, puedes entrenarlo en los datos preparados. Esto implica ajustar los parámetros del modelo para optimizar su rendimiento.
Es importante utilizar un conjunto de datos de validación para evaluar el rendimiento del modelo durante el entrenamiento. Esto te ayudará a evitar el sobreajuste.
5. Evaluación del modelo:

Una vez que el modelo esté entrenado, puedes evaluarlo en un conjunto de datos de prueba independiente. Esto te dará una idea de qué tan bien se generaliza el modelo a datos nuevos.
Si el rendimiento del modelo no es satisfactorio, puedes intentar ajustar los parámetros del modelo o probar un modelo diferente.
6. Implementación del modelo:

Una vez que estés satisfecho con el rendimiento del modelo, puedes implementarlo en producción. Esto significa que puedes usarlo para predecir el porcentaje de gasto en cada categoría para nuevos clientes.
Consideraciones adicionales:

Es importante tener en cuenta que este es un problema de aprendizaje supervisado, lo que significa que necesitas datos etiquetados para entrenar el modelo.
Si no tienes suficientes datos etiquetados, puedes considerar utilizar técnicas de aprendizaje semi-supervisado o no supervisado.
También es importante tener en cuenta que la precisión del modelo dependerá de la calidad de los datos.
Recursos adicionales:

Predicción del gasto del cliente
Aprendizaje automático para el análisis del gasto del cliente
Predicción del gasto futuro del cliente utilizando el aprendizaje automático
Espero que esta información te sea útil. Avísame si tienes alguna otra pregunta.

In [ ]:
# 1. Gráfico de líneas temporal por categoría y gasto
def line_plot_by_category(cc_num):
    df_filtered = df_filtrado[df_filtrado['cc_num'] == cc_num]
    fig = go.Figure()
    for category in df_filtered['category'].unique():
        df_category = df_filtered[df_filtered['category'] == category]
        fig.add_trace(go.Scatter(x=df_category['trans_date_trans_time'], y=df_category['amt'],
                                 mode='lines',
                                 name=category))
    fig.update_layout(title=f'Gasto a lo largo del tiempo por categoría para la cuenta {cc_num}',
                      xaxis_title='Fecha',
                      yaxis_title='Monto')
    fig.show()

# 2. Gráfico de barras apiladas de gasto por categoría
def stacked_bar_plot(cc_num):
    df_filtered = df_filtrado[df_filtrado['cc_num'] == cc_num]
    fig = go.Figure()
    for category in df_filtered['category'].unique():
        df_category = df_filtered[df_filtered['category'] == category]
        fig.add_trace(go.Bar(x=df_category['trans_date_trans_time'], y=df_category['amt'],
                             name=category))
    fig.update_layout(title=f'Gasto por categoría para la cuenta {cc_num}',
                      xaxis_title='Fecha',
                      yaxis_title='Monto',
                      barmode='stack')
    fig.show()

# 3. Gráfico de dispersión de gasto por fecha y hora
def scatter_plot(cc_num):
    df_filtered = df_filtrado[df_filtrado['cc_num'] == cc_num]
    fig = go.Figure()
    for category in df_filtered['category'].unique():
        df_category = df_filtered[df_filtered['category'] == category]
        fig.add_trace(go.Scatter(x=df_category['trans_date_trans_time'], y=df_category['amt'],
                                 mode='markers',
                                 name=category))
    fig.update_layout(title=f'Dispersión de gasto por fecha y hora para la cuenta {cc_num}',
                      xaxis_title='Fecha',
                      yaxis_title='Monto')
    fig.show()

# 4. Gráfico de torta de distribución de gasto por categoría
def pie_chart(cc_num):
    df_filtered = df_filtrado[df_filtrado['cc_num'] == cc_num]
    fig = go.Figure(data=[go.Pie(labels=df_filtered['category'], values=df_filtered['amt'])])
    fig.update_layout(title=f'Distribución de gasto por categoría para la cuenta {cc_num}')
    fig.show()

# 5. Heatmap de gasto por categoría y fecha
def heatmap(cc_num):
    df_filtered = df_filtrado[df_filtrado['cc_num'] == cc_num]
    pivot_df = df_filtered.pivot_table(index='trans_date_trans_time', columns='category', values='amt', aggfunc='sum')
    fig = go.Figure(data=go.Heatmap(z=pivot_df.values, x=pivot_df.columns, y=pivot_df.index))
    fig.update_layout(title=f'Heatmap de gasto por categoría y fecha para la cuenta {cc_num}',
                      xaxis_title='Categoría',
                      yaxis_title='Fecha')
    fig.show()

# 6. Gráfico de caja y bigotes de distribución de gasto
def box_plot(cc_num):
    df_filtered = df_filtrado[df_filtrado['cc_num'] == cc_num]
    fig = go.Figure()
    for category in df_filtered['category'].unique():
        df_category = df_filtered[df_filtered['category'] == category]
        fig.add_trace(go.Box(y=df_category['amt'], name=category))
    fig.update_layout(title=f'Distribución de gasto por categoría para la cuenta {cc_num}',
                      yaxis_title='Monto')
    fig.show()

# 7. Gráfico de barras de gasto promedio por hora del día
def bar_plot_hourly_avg(cc_num):
    df_filtered = df_filtrado[df_filtrado['cc_num'] == cc_num]
    df_filtered['hour'] = df_filtered['trans_date_trans_time'].dt.hour
    hourly_avg = df_filtered.groupby('hour')['amt'].mean().reset_index()
    fig = go.Figure(go.Bar(x=hourly_avg['hour'], y=hourly_avg['amt']))
    fig.update_layout(title=f'Gasto promedio por hora del día para la cuenta {cc_num}',
                      xaxis_title='Hora del día',
                      yaxis_title='Gasto promedio')
    fig.show()

In [ ]:
df_filtrado["cc_num"].unique()

array([   4642255475285942, 4512828414983801773,      30270432095985,
          4364010865167176,    6011438889172900,    4586810168620942,
           344709867813900,    6538891242532018,    4904681492230012,
          6538441737335434], dtype=int64)

In [ ]:
# Ejemplo de cómo utilizar las funciones:
cc_num_ejemplo = 4642255475285942
line_plot_by_category(cc_num_ejemplo)
stacked_bar_plot(cc_num_ejemplo)
scatter_plot(cc_num_ejemplo)
pie_chart(cc_num_ejemplo)
heatmap(cc_num_ejemplo)
box_plot(cc_num_ejemplo)
bar_plot_hourly_avg(cc_num_ejemplo)

AttributeError: Can only use .dt accessor with datetimelike values

In [ ]:
# Guardar y abrir modelos
from tensorflow.keras.models import load_model

In [ ]:
load_model("../data/modelo_4642255475285942.keras")

c:\Users\enaat\.conda\envs\ecomovilidad\Lib\site-packages\keras\src\saving\saving_lib.py:415: UserWarning: Skipping variable loading for optimizer 'adam', because it has 18 variables whereas the saved optimizer has 17 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


ValueError: A total of 4 objects could not be loaded. Example error message for object <Dense name=dense_20, built=True>:

Layer 'dense_20' expected 2 variables, but received 0 variables during loading. Expected: ['kernel', 'bias']

List of objects that could not be loaded:
[<Dense name=dense_20, built=True>, <Dense name=dense_21, built=True>, <Dense name=dense_22, built=True>, <Dense name=dense_23, built=True>]

In [ ]:
cuenta = 4642255475285942

In [ ]:
import pickle

In [ ]:
def predicion(df):
    with open(f'../data/modelo_{cuenta}.pkl', 'rb') as file:
        modelo_pkl = pickle.load(file)

    instancia = df["amt"][-10: -1].to_list()
    instancia = np.array(instancia).reshape(-1, 10, 1)
    prediccion_gasto = modelo_pkl.predict(instancia)
    prediccion_gasto = prediccion_gasto[0][0]

    return prediccion_gasto

In [ ]:
def funcion_prediccion_gastos(num_meses):
    with open(f'../data/modelo_{cuenta}.pkl', 'rb') as file:
        modelo_pkl = pickle.load(file)
    #recorremos un bucle para predecir la primera instancia, agregar esta a una nueva lista y continuar prediciendo en función de las semanas indicadas.    
    for n in range(num_meses*31):
        if n == 0:
            #se hace la primera prediccion y se appe
            prediccion_gasto = predicion(df) 
            instancia = df["amt"][-9: -1].to_list()
            instancia.append(prediccion_gasto)

        else:

            instancia_array = np.array(instancia).reshape(-1, 10, 1)
            prediccion_gasto = modelo_pkl.predict(instancia_array)
            prediccion_gasto = prediccion_gasto[0][0]
            instancia = instancia[1:]
            instancia.append(prediccion_gasto)

    return instancia[(len(instancia)-num_meses):]

In [ ]:
def funcion_gastos(df):

    fig = px.line(data_frame=df, 
                  x="trans_date_trans_time", 
                  y="amt")
    fig.show()

In [ ]:
df = pd.read_csv("../data/datos.csv")
df = df[df["cc_num"] == cuenta]

In [ ]:
funcion_prediccion_gastos(1)

ModuleNotFoundError: No module named 'dill'

In [ ]:
import keras

keras.__version__

'3.3.3'